In [1]:
import pandas as pd
import psycopg2 as pg
import geopandas as gpd

from masterplan_tools.City_model.city_model import CityModel
from masterplan_tools.Provision_getter.provision_getter import ProvisionModel
from masterplan_tools.Balancer.balancer import MasterPlan

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [5]:
engine = pg.connect("dbname='city_db_final' user='postgres' host='10.32.1.107' port='5432' password='postgres'")
service_groups = ["schools", "kindergartens"]
matrix = pd.read_pickle("/home/gk/jupyter/MMG_SPB.pkl")

city_model = CityModel(
    city_name="Санкт-Петербург",
    city_crs=32636,
    city_admin_level=5,
    service_names=service_groups,
    accessibility_matrix=matrix,
    city_id=1,
    engine=engine,
)

In [7]:
city_model.collect_data()

2023-05-14 19:15:37.264 | INFO     | mp_tools.Data_getter.data_getter:get_city_geometry:63 - City geometry is not provided. Getting water geometry from OSM via overpass turbo
/home/gk/.local/lib/python3.8/site-packages/osm2geojson/main.py:514: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for line in merged_line:
2023-05-14 19:15:38.941 | INFO     | mp_tools.Data_getter.data_getter:get_city_geometry:77 - Got city geometry
2023-05-14 19:15:38.942 | INFO     | mp_tools.Data_getter.data_getter:get_roads_geometry:162 - Roads geometries are not provided. Getting roads geometries from OSM via OSMNX
2023-05-14 19:16:51.284 | INFO     | mp_tools.Data_getter.data_getter:get_roads_geometry:177 - Got roads geometries
2023-05-14 19:16:51.286 | INFO     | mp_tools.Data_getter.data_getter:get_railways_geometry:129 - Railways geometries are not provi

In [2]:
services_accessabilities = {
    "kindergartens": 4,
    "schools": 7,
    "universities": 60,
    "hospitals": 60,
    "policlinics": 13,
    "theaters": 60,
    "cinemas": 60,
    "cafes": 30,
    "bakeries": 30,
    "fastfoods": 30,
    "music_school": 30,
    "sportgrounds": 7,
    "swimming_pools": 30,
    "conveniences": 8,
    "recreational_areas": 25,
    "pharmacies": 7,
    "playgrounds": 2,
    "supermarkets": 30,
}

standard_dict = {
    "kindergartens": 61,
    "schools": 120,
    "universities": 13,
    "hospitals": 9,
    "policlinics": 27,
    "theaters": 5,
    "cinemas": 10,
    "cafes": 72,
    "bakeries": 72,
    "fastfoods": 72,
    "music_school": 8,
    "sportgrounds": 15,
    "swimming_pools": 50,
    "conveniences": 90,
    "recreational_areas": 5000,
    "pharmacies": 50,
    "playgrounds": 550,
    "supermarkets": 992,
}


for service_type in city_model.service_names:
    service_gdf = get_service(engine=engine, city_crs=city_crs, city_id=city_id, service_type=service_type)

    model = ProvisionModel(
        SUB_GROUP=service_type,
        blocks=blocks_gdf,
        matrix=matrix,
        buildings=buildings_gdf,
        standard=standard_dict[service_type],
        ACCS_TIME=services_accessabilities[service_type],
        service_gdf=service_gdf,
    )

    if service_type == "schools":
        school = model.run()

    elif service_type == "kindergartens":
        kindergarten = model.run()

    # elif service_type == 'recreational_areas':
    #     greening = model.run()

greening = gpd.read_parquet("./output_data/recreational_areas.parquet")
# prov_data = blocks_gdf.merge(school[['id', 'population_unprov_schools']], left_on='id', right_on='id').merge(kindergarten[['id', 'population_unprov_kindergartens']],
# left_on='id', right_on='id').merge(greening[['id', 'population_unprov_recreational_areas']], left_on='id', right_on='id')

/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  0%|          | 0/605 [00:00<?, ?it/s]

количество кварталов c сервисом schools: 51
количество жилых кварталов: 1135
количество кварталов всего: 1609
количество кварталов c ошибкой: 0


/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


  0%|          | 0/691 [00:00<?, ?it/s]

количество кварталов c сервисом kindergartens: 29
количество жилых кварталов: 765
количество кварталов всего: 900
количество кварталов c ошибкой: 0


In [5]:
school.loc[school[f"population_unprov_schools"] > 0]

,id,geometry,provision_schools,id_schools,population_prov_schools,population_unprov_schools,population
566,566,"POLYGON ((320918.941 6654703.825, 320918.860 6...",25,327,297,861,1159
571,571,"POLYGON ((320624.168 6654912.212, 320624.085 6...",0,0,0,811,811
782,782,"POLYGON ((324402.745 6643448.134, 325085.481 6...",81,782,6733,1566,8300
991,991,"POLYGON ((331530.345 6664441.716, 331530.553 6...",53,985,340,300,641
997,997,"POLYGON ((331431.154 6664712.267, 331427.433 6...",0,0,0,26,26
...,...,...,...,...,...,...,...
5916,5916,"POLYGON ((364484.991 6632347.945, 364404.695 6...",29,5916,1466,3550,5017
5925,5925,"POLYGON ((364022.684 6633357.696, 364022.750 6...",57,5925,6716,5066,11783
5988,5988,"POLYGON ((360824.109 6651704.475, 360824.113 6...",64,5980,2363,1309,3672
6004,6004,"POLYGON ((365734.620 6623835.369, 365726.015 6...",70,6005,5192,2212,7405


In [7]:
# school = gpd.read_parquet('./output_data/schools.parquet')
# kindergarten = gpd.read_parquet('./output_data/kindergartens.parquet')
# greening = gpd.read_parquet('./output_data/recreational_areas.parquet')

In [8]:
polygon = gpd.read_file("./output_data/polygon.geojson")
polygon.explore()

In [9]:
gdf = gpd.read_parquet("/Users/mvin/Desktop/masterplanning/output_data/BLOCKS_AND_BUILDINGS_INFO-2.parquet")

In [10]:
gdf.head()

,geometry,block_id,current_population,floors,current_living_area,current_green_capacity,current_green_area,current_parking_capacity,current_industrial_area,area
0,"POLYGON ((316983.653 6644038.696, 316986.485 6...",0,0.0,0.0,0.0000,1091.0,219008.0,237.0,1208.07200,2.738972e+06
1,"POLYGON ((315287.843 6646411.375, 315288.850 6...",1,671.0,1.0,14106.1824,452.0,13070.0,0.0,16625.23190,2.795374e+06
2,"POLYGON ((315582.670 6646809.216, 315582.581 6...",2,48.0,1.0,1090.8619,0.0,0.0,0.0,319.27800,1.771556e+04
3,"POLYGON ((315264.235 6646596.731, 315274.102 6...",3,10.0,1.5,172.0790,0.0,0.0,0.0,1029.45610,2.680521e+05
4,"POLYGON ((313479.271 6646628.746, 313479.599 6...",4,47.0,1.0,1112.1592,0.0,0.0,145.0,73477.11422,2.380600e+06


In [11]:
def balanc_data(gdf, polygon, school, kindergarten, greening):

    Hectare = 10000
    intersecting_blocks = gpd.overlay(gdf, polygon, how="intersection").drop(columns=["id"])
    intersecting_blocks.rename(columns={"id_1": "id"}, inplace=True)
    gdf = intersecting_blocks

    gdf["current_building_area"] = gdf["current_living_area"] + gdf["current_industrial_area"]
    gdf_ = gdf[
        [
            "block_id",
            "area",
            "current_living_area",
            "current_industrial_area",
            "current_population",
            "current_green_area",
            "floors",
        ]
    ]

    gdf_ = (
        gdf_.merge(school[["id", "population_unprov_schools"]], left_on="block_id", right_on="id")
        .merge(kindergarten[["id", "population_unprov_kindergartens"]], left_on="block_id", right_on="id")
        .merge(greening[["id", "population_unprov_recreational_areas"]], left_on="block_id", right_on="id")
    )
    gdf_.drop(["id_x", "id_y", "id"], axis=1, inplace=True)

    gdf_["area"] = gdf_["area"] / Hectare
    gdf_["current_living_area"] = gdf_["current_living_area"] / Hectare
    gdf_["current_industrial_area"] = gdf_["current_industrial_area"] / Hectare
    gdf_["current_green_area"] = gdf_["current_green_area"] / Hectare

    df_sum = gdf_.sum()
    df_sum["floors"] = gdf_["floors"].mean()
    df_new = pd.DataFrame(df_sum).T

    sample = df_new[df_new["area"] > 10].sample()
    sample = sample.to_dict("records")
    block = sample[0].copy()

    return block

In [12]:
block = balanc_data(gdf=gdf, polygon=polygon, school=school, kindergarten=kindergarten, greening=greening)
block

{'block_id': 140932.0,
 'area': 270.7060423617665,
 'current_living_area': 58.46889635,
 'current_industrial_area': 50.056015810199995,
 'current_population': 49734.0,
 'current_green_area': 17.376400000000004,
 'floors': 3.2708333333333335,
 'population_unprov_schools': 1240.0,
 'population_unprov_kindergartens': 0.0,
 'population_unprov_recreational_areas': 14336.0}

In [13]:
mp = MasterPlan(
    area=block["area"],
    current_living_area=block["current_living_area"],
    current_industrial_area=block["current_industrial_area"],
    current_population=block["current_population"],
    current_green_area=block["current_green_area"],
    current_unprov_schoolkids=block["population_unprov_schools"],
    current_unprov_kids=block["population_unprov_kindergartens"],
    current_unprov_green_population=block["population_unprov_recreational_areas"],
)
updated_block_info = mp.optimal_solution_indicators()
# updated_block_info["block_id"] = '138'
updated_block_info

/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = self.results.append(temp_result, ignore_index=True)
/Users/mvin/Desktop/masterplanning/Balancer/balancer.py:162: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.results = 

{'area': 270.7060423617665,
 'population': 76009.0,
 'b': 29.99601356107917,
 'green_coef_G': 9.030707660260864,
 'living_area': 137.28342198173553,
 'schools_area': 5.4,
 'schools_capacity': 3300,
 'kindergartens_area': 6.6,
 'kindergartens_capacity': 1680,
 'green_area': 54.0510068790854,
 'G_min_capacity': 90085.01146514234,
 'G_max_capacity': 45042.50573257117,
 'green_coef_G_capacity': 59852.459976014834,
 'op_area': 0.078825,
 'parking1_area': 19.8639,
 'parking2_area': 19.312124999999998}

In [15]:
school.loc[2157, :]

id                                                                        2157
geometry                     POLYGON ((340408.2947395761 6639979.697077261,...
provision_schools                                                          100
id_schools                                                                2157
population_prov_schools                                                  30140
population_unprov_schools                                                    0
population                                                               30140
Name: 2157, dtype: object

In [16]:
kindergarten.loc[2157, :]

id                                                                              2157
geometry                           POLYGON ((340408.2947395761 6639979.697077261,...
provision_kindergartens                                                          100
id_kindergartens                                                                2157
population_prov_kindergartens                                                  30140
population_unprov_kindergartens                                                    0
population                                                                     30140
Name: 2157, dtype: object

In [17]:
updated_block = {
    "area": 269.91357042657916,
    "population": 49547.0,
    "b": 24.000000000000004,
    "green_coef_G": 9.0,
    "living_area": 63.8661511,
    "schools_area": 4.7,
    "schools_capacity": 2500,
    "kindergartens_area": 4.4,
    "kindergartens_capacity": 1120,
    "green_area": 43.0201,
    "G_min_capacity": 71700.16666666667,
    "G_max_capacity": 35850.083333333336,
    "green_coef_G_capacity": 47800.11111111111,
    "op_area": 0.061914000000000004,
    "parking1_area": 15.602328000000002,
    "parking2_area": 15.16893,
    "block_id": 2157,
}

In [18]:
for service_type in GROUPS:
    service_gdf = get_service(
        engine=engine,
        city_crs=city_crs,
        city_id=city_id,
        service_type=service_type,
    )

    model = ProvisionModel(
        SUB_GROUP=service_type,
        blocks=blocks_gdf,
        matrix=matrix,
        buildings=buildings_gdf,
        standard=standard_dict[service_type],
        ACCS_TIME=services_accessabilities[service_type],
        service_gdf=service_gdf,
        updated_block_info=updated_block,
    )

    if service_type == "schools":
        school = model.run()

    elif service_type == "kindergartens":
        kindergarten = model.run()

/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


3972
6472


  0%|          | 0/605 [00:00<?, ?it/s]

количество кварталов c сервисом schools: 51
количество жилых кварталов: 1135
количество кварталов всего: 1609
количество кварталов c ошибкой: 0


/Users/mvin/Desktop/mp_dev/mp_dev/lib/python3.9/site-packages/geopandas/io/sql.py:158: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(


2406
3526


  0%|          | 0/691 [00:00<?, ?it/s]

количество кварталов c сервисом kindergartens: 29
количество жилых кварталов: 765
количество кварталов всего: 900
количество кварталов c ошибкой: 0


In [19]:
school.loc[2157, :]

id                                                                        2157
geometry                     POLYGON ((340408.2947395761 6639979.697077261,...
provision_schools                                                          100
id_schools                                                                2157
population_prov_schools                                                  49547
population_unprov_schools                                                    0
population                                                               49547
Name: 2157, dtype: object

In [20]:
kindergarten.loc[2157, :]

id                                                                              2157
geometry                           POLYGON ((340408.2947395761 6639979.697077261,...
provision_kindergartens                                                          100
id_kindergartens                                                                2157
population_prov_kindergartens                                                  49547
population_unprov_kindergartens                                                    0
population                                                                     49547
Name: 2157, dtype: object